In [1]:
import numpy as np
from random import random
inputs = np.array([[random()/2 for i in range(2)] for j in range(1000)])
inputs[:50]

array([[0.12719392, 0.07602623],
       [0.42571454, 0.44914012],
       [0.05950468, 0.05146188],
       [0.31472534, 0.23648752],
       [0.061582  , 0.17775878],
       [0.26516871, 0.21099376],
       [0.07276815, 0.2966436 ],
       [0.12780171, 0.40894899],
       [0.26000584, 0.40420933],
       [0.08128355, 0.3214042 ],
       [0.13140583, 0.42142161],
       [0.24875974, 0.23672868],
       [0.38391742, 0.36774915],
       [0.08721717, 0.1845068 ],
       [0.41734677, 0.38446297],
       [0.2489276 , 0.12474914],
       [0.07660528, 0.13836602],
       [0.28749524, 0.08352194],
       [0.2930273 , 0.19112219],
       [0.3471332 , 0.09847199],
       [0.32795873, 0.08230438],
       [0.21339041, 0.16101675],
       [0.36194694, 0.01171719],
       [0.32423601, 0.47362081],
       [0.45483277, 0.01995599],
       [0.1326681 , 0.4759635 ],
       [0.2183375 , 0.2953861 ],
       [0.20994714, 0.10284431],
       [0.26467161, 0.10438666],
       [0.01780976, 0.11341754],
       [0.

In [2]:
layers = [2,7,1]
weight = []
for i in range(len(layers)-1):
    w = np.random.randn(layers[i] , layers[i+1] )
    #w = np.array([[random()/2 for _ in range(layers[i+1])] for _ in range(layers[i])])
    weight.append(w)
weight

[array([[ 0.15485921, -0.87631017, -1.00534223,  0.64501655,  0.81029146,
          0.95863109, -1.09586384],
        [ 0.05261376,  0.86886353, -1.09540752,  0.79323226, -1.14472996,
          0.08420204, -1.01230927]]), array([[ 0.41871381],
        [-0.44755161],
        [-0.71574782],
        [-0.34354989],
        [-0.32074018],
        [-1.54785278],
        [-1.00916125]])]

In [51]:
'''
Stochastic Training of Neural Networks via
Successive Convex Approximation
'''
import numpy as np
from random import random

class MLP():

    def __init__(self, num_inputs=3, num_hidden=[3,5], num_outputs=3, weight = 0):

        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.weights = weight
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        '''
        weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i] , layers[i+1] )
            weights.append(w)
        self.weights = weights
        # print("weights: ", weights)
        '''
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations
        # print("activations: ", activations)

        derivatives = []
        for i in range(len(layers)-1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives
        # print("derivatives: ", derivatives)

    def forward(self, inputs):
        
        activations = inputs
        self.activations[0] = activations
        for i, w in enumerate(self.weights):
            net_inputs = np.dot(activations, w)
            # print("net inputs:", net_inputs)
            activations = self.tanh(net_inputs)
            self.activations[i+1] = activations           
            # print("activations: {} ".format(activations))
            # print("final activations:", activations)
        return activations


    def back_prop(self, error):
        for i in reversed(range(len(self.derivatives))):
            activations = self.activations[i+1]
            #print("activations:", activations)
            # delta:= error.dedrivative_of_activation_function
            delta = error * self.tanh_derivative(activations)
            #print("delta: ", delta)
            delta_reshaped = delta.reshape(delta.shape[0], -1).T
            #print("delta reshaped:" , delta_reshaped)
            current_activations = self.activations[i]
            current_activations_reshaped = current_activations.reshape(current_activations.shape[0],-1)
            #print("current activations:", current_activations)
            #print("current activations reshaped", current_activations_reshaped)
            self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
            # error := error. derivative_of_activation_function.weights
            # delta := error. derivative_of_activation_function
            # error := delta.weights
            error = np.dot(delta, self.weights[i].T)
            #print(error)
            #print("derivatives:", self.derivatives[i])
        return error

    def training(self, inputs, targets, epochs, learning_rate):
        for i in range(epochs):
            sum_error = 0
            for input, target in zip(inputs, targets):

                output = self.forward(input)
                error = output - target
                self.back_prop(error)
                self.gradient_descent(learning_rate)
                sum_error += self.mean_square_error(target, output)
            print("Error: {} at Epoch {}".format(sum_error / len(inputs), i+1))

    def mean_square_error(self, target, output):
        return np.average((output - target)**2)

    def gradient_descent(self, learning_rate):
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights -= derivatives * learning_rate 


    def tanh(self, x):
        return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))

    def tanh_derivative(self, x):
        return (1-x**2)
   

if __name__ == "__main__":
    mlp = MLP(2,[7],1, weight)
    #inputs = np.random.rand(mlp.num_inputs)
    #inputs = np.array([1, 2, 3])
    #inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in inputs])
    mlp.training(inputs, targets, 100, .8)
    pre_input = np.array([0.3, 0.4])
    prediction = mlp.forward(pre_input)
    #targets = np.array([6])
    print("Withoud Batch Our Neural Network Shows: {} + {} = {}".format(pre_input[0], pre_input[1], prediction[0]))
    #print("the network input is: {}" .format(inputs)) 
    #print("the network output is: {}" .format(outputs)) 
    


Error: 0.0002711767200777975 at Epoch 1
Error: 0.0002711782242260903 at Epoch 2
Error: 0.0002711797140411572 at Epoch 3
Error: 0.0002711811894543877 at Epoch 4
Error: 0.00027118265041275704 at Epoch 5
Error: 0.00027118409687776037 at Epoch 6
Error: 0.0002711855288244031 at Epoch 7
Error: 0.00027118694624025865 at Epoch 8
Error: 0.0002711883491245839 at Epoch 9
Error: 0.00027118973748748565 at Epoch 10
Error: 0.000271191111349144 at Epoch 11
Error: 0.00027119247073907886 at Epoch 12
Error: 0.0002711938156954606 at Epoch 13
Error: 0.0002711951462644716 at Epoch 14
Error: 0.00027119646249969785 at Epoch 15
Error: 0.0002711977644615628 at Epoch 16
Error: 0.00027119905221679585 at Epoch 17
Error: 0.0002712003258379338 at Epoch 18
Error: 0.0002712015854028522 at Epoch 19
Error: 0.00027120283099432984 at Epoch 20
Error: 0.00027120406269963383 at Epoch 21
Error: 0.0002712052806101364 at Epoch 22
Error: 0.00027120648482095613 at Epoch 23
Error: 0.0002712076754306148 at Epoch 24
Error: 0.0002712

In [50]:
pre_input = np.array([0.3, 0.5])
prediction = mlp.forward(pre_input)
    #targets = np.array([6])
print("Without Batch Our Neural Network Shoews: {} + {} = {}".format(pre_input[0], pre_input[1], prediction[0]))

Without Batch Our Neural Network Shoews: 0.3 + 0.5 = 0.7920696714836456


In [52]:
'''
Stochastic Training of Neural Networks via
Successive Convex Approximation
'''
import numpy as np
from random import random
from matplotlib import pyplot as plt
%matplotlib inline

class MLP_B():

    def __init__(self, num_inputs=3, num_hidden=[3,5], num_outputs=3, weight = 0):

        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.weights = weight
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        '''
        weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i] , layers[i+1] )
            weights.append(w)
        self.weights = weights
        # print("weights: ", weights)
        '''
        
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations
        # print("activations: ", activations)

        derivatives = []
        for i in range(len(layers)-1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives
        # print("derivatives: ", derivatives)

    def forward(self, inputs):
        
        activations = inputs
        self.activations[0] = activations
        for i, w in enumerate(self.weights):
            net_inputs = np.dot(activations, w)
            # print("net inputs:", net_inputs)
            activations = self.tanh(net_inputs)
            self.activations[i+1] = activations           
            # print("activations: {} ".format(activations))
            # print("final activations:", activations)
        return activations


    def back_prop(self, error):
        for i in reversed(range(len(self.derivatives))):
            activations = self.activations[i+1]
            #print("activations:", activations)
            # delta:= error.dedrivative_of_activation_function
            delta = error * self.tanh_derivative(activations)
            #print("delta: ", delta)
            delta_reshaped = delta.reshape(delta.shape[0], -1).T
            #print("delta reshaped:" , delta_reshaped)
            current_activations = self.activations[i]
            current_activations_reshaped = current_activations.reshape(current_activations.shape[0],-1)
            #print("current activations:", current_activations)
            #print("current activations reshaped", current_activations_reshaped)
            self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
            # error := error. derivative_of_activation_function.weights
            # delta := error. derivative_of_activation_function
            # error := delta.weights
            error = np.dot(delta, self.weights[i].T)
            #print(error)
            #print("derivatives:", self.derivatives[i])
        return error

    def training(self, inputs, targets, epochs, learning_rate):
        for i in range(epochs):
            #sum_error = 0
            batch_size = 100
            for step in range(int(len(inputs)/batch_size)):
                sum_error = 0
                first_index = 0
                last_index = batch_size
                batch_input = inputs[first_index: last_index]
                batch_target = targets[first_index: last_index]

                for input, target in zip(batch_input, batch_target):
                    output = self.forward(input)
                    error = output - target
                    self.back_prop(error)
                    self.gradient_descent(learning_rate)
                    sum_error += self.mean_square_error(target, output)
                    
                first_index = last_index
                last_index += batch_size 
                #print("Error: {} at batch {}".format(sum_error / batch_size, step+1))
               
                
  
            print("Error: {} at Epoch {}".format(sum_error / len(inputs), i+1))
            #plt.plot(sum_error)

    def mean_square_error(self, target, output):
        return np.average((output - target)**2)

    def gradient_descent(self, learning_rate):
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights -= derivatives * learning_rate 


    def tanh(self, x):
        return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))

    def tanh_derivative(self, x):
        return (1-x**2)
   


if __name__ == "__main__":
    mlp_b = MLP_B(2,[7],1 ,weight )
    #inputs = np.random.rand(mlp.num_inputs)
    #inputs = np.array([1, 2, 3])
    #inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in inputs])
    mlp_b.training(inputs, targets, 100, .8)
    pre_input = np.array([0.3, 0.4])
    prediction = mlp_b.forward(pre_input)
    #targets = np.array([6])
    print("Using Batch Our Neural network Shows that {} + {} = {}".format(pre_input[0], pre_input[1], prediction[0]))
    #print("the network input is: {}" .format(inputs)) 
    #print("the network output is: {}" .format(outputs)) 
  


Error: 1.6552613527915124e-05 at Epoch 1
Error: 1.6517013689356568e-05 at Epoch 2
Error: 1.6501963565660943e-05 at Epoch 3
Error: 1.649663647180579e-05 at Epoch 4
Error: 1.6497397947959167e-05 at Epoch 5
Error: 1.650146876418067e-05 at Epoch 6
Error: 1.6507072275678335e-05 at Epoch 7
Error: 1.6513142093019418e-05 at Epoch 8
Error: 1.6519071606730672e-05 at Epoch 9
Error: 1.652454275947671e-05 at Epoch 10
Error: 1.652941391071914e-05 at Epoch 11
Error: 1.6533647671644207e-05 at Epoch 12
Error: 1.6537265175545626e-05 at Epoch 13
Error: 1.65403176350816e-05 at Epoch 14
Error: 1.6542869085451796e-05 at Epoch 15
Error: 1.6544986279279207e-05 at Epoch 16
Error: 1.654673308712629e-05 at Epoch 17
Error: 1.6548167683195442e-05 at Epoch 18
Error: 1.6549341408716128e-05 at Epoch 19
Error: 1.655029860855895e-05 at Epoch 20
Error: 1.6551076999613694e-05 at Epoch 21
Error: 1.6551708299652733e-05 at Epoch 22
Error: 1.65522189544513e-05 at Epoch 23
Error: 1.65526308699691e-05 at Epoch 24
Error: 1.6552

In [53]:
pre_input = np.array([0.30,  0.3])
prediction_b = mlp_b.forward(pre_input)
    #targets = np.array([6])
print("With Batch Our Neural network Shows that {} + {} = {}".format(pre_input[0], pre_input[1], prediction_b[0]))

With Batch Our Neural network Shows that 0.3 + 0.3 = 0.6059366340992841


In [33]:
'''
Stochastic Training of Neural Networks via
Successive Convex Approximation
'''
import numpy as np
from random import random

class MLP():

    def __init__(self, num_inputs=3, num_hidden=[3,5], num_outputs=3, weight = 0):

        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.weights = weight
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        '''
        weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i] , layers[i+1] )
            weights.append(w)
        self.weights = weights
        # print("weights: ", weights)
        '''
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations
        # print("activations: ", activations)

        derivatives = []
        for i in range(len(layers)-1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives
        # print("derivatives: ", derivatives)

    def forward(self, inputs):
        
        activations = inputs
        self.activations[0] = activations
        for i, w in enumerate(self.weights):
            net_inputs = np.dot(activations, w)
            # print("net inputs:", net_inputs)
            activations = self.tanh(net_inputs)
            self.activations[i+1] = activations           
            # print("activations: {} ".format(activations))
            # print("final activations:", activations)
        return activations


    def back_prop(self, error):
        for i in reversed(range(len(self.derivatives))):
            activations = self.activations[i+1]
            #print("activations:", activations)
            # delta:= error.dedrivative_of_activation_function
            delta = error * self.tanh_derivative(activations)
            #print("delta: ", delta)
            delta_reshaped = delta.reshape(delta.shape[0], -1).T
            #print("delta reshaped:" , delta_reshaped)
            current_activations = self.activations[i]
            current_activations_reshaped = current_activations.reshape(current_activations.shape[0],-1)
            #print("current activations:", current_activations)
            #print("current activations reshaped", current_activations_reshaped)
            self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
            # error := error. derivative_of_activation_function.weights
            # delta := error. derivative_of_activation_function
            # error := delta.weights
            error = np.dot(delta, self.weights[i].T)
            #print(error)
            #print("derivatives:", self.derivatives[i])
        return error

    def training(self, inputs, targets, epochs, learning_rate):
        for i in range(epochs):
            sum_error = 0
            for input, target in zip(inputs, targets):

                output = self.forward(input)
                error = output - target
                self.back_prop(error)
                self.gradient_descent(learning_rate)
                sum_error += self.mean_square_error(target, output)
            #print("Error: {} at Epoch {}".format(sum_error / len(inputs), i+1))
    
    def training_batch(self, inputs, targets, epochs, learning_rate):
        for i in range(epochs):
            #sum_error = 0
            batch_size = 100
            for step in range(int(len(inputs)/batch_size)):
                sum_error = 0
                first_index = 0
                last_index = batch_size
                batch_input = inputs[first_index: last_index]
                batch_target = targets[first_index: last_index]

                for input, target in zip(batch_input, batch_target):
                    output = self.forward(input)
                    error = output - target
                    self.back_prop(error)
                    self.gradient_descent(learning_rate)
                    sum_error += self.mean_square_error(target, output)
                    
                first_index = last_index
                last_index += batch_size 
                #print("Error: {} at batch {}".format(sum_error / batch_size, step+1))
               
                
  
            #print("Error: {} at Epoch {}".format(sum_error / len(inputs), i+1))
            #plt.plot(sum_error)

    def mean_square_error(self, target, output):
        return np.average((output - target)**2)

    def gradient_descent(self, learning_rate):
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights -= derivatives * learning_rate 


    def tanh(self, x):
        return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))

    def tanh_derivative(self, x):
        return (1-x**2)
   

if __name__ == "__main__":
    mlp = MLP(2,[7],1, weight)
    #inputs = np.random.rand(mlp.num_inputs)
    #inputs = np.array([1, 2, 3])
    #inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in inputs])
    mlp.training(inputs, targets, 100, .8)
    pre_input = np.array([0.3, 0.4])
    prediction = mlp.forward(pre_input)
    #targets = np.array([6])
    print("without Batch Our Neural Network Shows: {} + {} = {}".format(pre_input[0], pre_input[1], prediction[0]))
    #print("the network input is: {}" .format(inputs)) 
    #print("the network output is: {}" .format(outputs)) 
    mlp_b = MLP(2,[7],1 ,weight )
    #inputs = np.random.rand(mlp.num_inputs)
    #inputs = np.array([1, 2, 3])
    #inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in inputs])
    mlp_b.training_batch(inputs, targets, 100, .8)
    pre_input = np.array([0.3, 0.4])
    prediction = mlp_b.forward(pre_input)
    #targets = np.array([6])
    print("Using Batch Our Neural Network Shows: {} + {} = {}".format(pre_input[0], pre_input[1], prediction[0]))
    #print("the network input is: {}" .format(inputs)) 
    #print("the network output is: {}" .format(outputs)) 
    


without Batch Our Neural Network Shows: 0.3 + 0.4 = 0.7155102131769498
Using Batch Our Neural Network Shows: 0.3 + 0.4 = 0.6996198784340236


In [66]:
import numpy as np
import pandas as pd
df = pd.read_csv("Data_SCA.csv")

np.random.seed(1)
m = np.zeros(7)
type(df.to_numpy())

numpy.ndarray

In [3]:
n=[3]
m=[3,5]
p=[2]

In [4]:
#layers = [n]+m+[p]
layers = n+m+p
print(layers)

[3, 3, 5, 2]


In [7]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd

In [6]:
from matplotlib import pyplot as plt
data = [ [2,  3,  5 ],
        [3, 4, 7],
        [3, 6, 9 ],
       [1, 3, 4]]

In [8]:
import numpy as np


class MLP(object):

    def __init__(self, num_inputs=7, num_hidden=[3,5], num_outputs=2):

        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        self.weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i], layers[i+1])
            self.weights.append(w)

    def forward_propagate(self,inputs):
        activations = inputs
        for w in self.weights:
            net_inputs = np.dot(activations, w)
            activations = self.sigmoid(net_inputs)
        return activations

    def sigmoid(self, x):
        return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))

if __name__ == "__main__":
    mlp = MLP()
    inputs = np.random.randn(mlp.num_inputs)
    outputs = mlp.forward_propagate(inputs)
    print("the network input is: {}" .format(inputs)) 
    print("the network input is: {}" .format(outputs)) 



the network input is: [ 0.83898341  0.93110208  0.28558733  0.88514116 -0.75439794  1.25286816
  0.51292982]
the network input is: [0.37622306 0.99751563]


In [19]:
items = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])


In [2]:
n = np.random.randn(10)
#print (n)
plt.plot(n, 'o')

NameError: name 'plt' is not defined

In [5]:
inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
inputs

array([[0.15360698, 0.11081771],
       [0.01687841, 0.39278484],
       [0.37963109, 0.37671021],
       ...,
       [0.03589354, 0.18585574],
       [0.22705925, 0.1523342 ],
       [0.12052998, 0.3102365 ]])

In [10]:
'''  
layers = [2,7,1] 
weight = []
for i in range(len(layers)-1):
    w = np.random.randn(layers[i] , layers[i+1] )
    weight.append(w)
'''  
if __name__ == "__main__":
    mlp = MLP(2,[7],1,weight)
    #inputs = np.random.rand(mlp.num_inputs)
    #inputs = np.array([1, 2, 3])
    #inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in inputs])
    mlp.training(inputs, targets, 100, .8)
    pre_input = np.array([0.3, 0.2])
    prediction = mlp.forward(pre_input)
    #targets = np.array([6])
    print("{} + {} = {}".format(pre_input[0], pre_input[1], prediction[0]))
    #print("the network input is: {}" .format(inputs)) 
    #print("the network output is: {}" .format(outputs)) 

Error: 0.0002650680037038177 at Epoch 1
Error: 0.00026498336062642086 at Epoch 2
Error: 0.00026489952626471895 at Epoch 3
Error: 0.00026481649125714534 at Epoch 4
Error: 0.0002647342464086478 at Epoch 5
Error: 0.000264652782681403 at Epoch 6
Error: 0.0002645720911862581 at Epoch 7
Error: 0.0002644921631748544 at Epoch 8
Error: 0.0002644129900323846 at Epoch 9
Error: 0.00026433456327093886 at Epoch 10
Error: 0.0002642568745234132 at Epoch 11
Error: 0.00026417991553792034 at Epoch 12
Error: 0.00026410367817268653 at Epoch 13
Error: 0.0002640281543913957 at Epoch 14
Error: 0.00026395333625893973 at Epoch 15
Error: 0.00026387921593755734 at Epoch 16
Error: 0.00026380578568332985 at Epoch 17
Error: 0.00026373303784300406 at Epoch 18
Error: 0.0002636609648511177 at Epoch 19
Error: 0.0002635895592274073 at Epoch 20
Error: 0.00026351881357447517 at Epoch 21
Error: 0.0002634487205756929 at Epoch 22
Error: 0.0002633792729933222 at Epoch 23
Error: 0.00026331046366683534 at Epoch 24
Error: 0.00026

In [53]:
import numpy as np
from random import random

class MLP():

    def __init__(self, num_inputs=2, num_hidden=[3,5], num_outputs=2):

        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i] , layers[i+1])
            weights.append(w)
        self.weights = weights

        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations

        derivatives = []
        for i in range(len(layers)-1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives
        for i in reversed(range(len(self.derivatives))):
            print (i)
        
        

    def forward(self, inputs):
       
        activations = inputs
        self.activations[0] = activations
        for i, w in enumerate(self.weights):
            net_inputs = np.dot(activations, w)
            activations = self.tanh(net_inputs)
            self.activations[i+1] = activations           
        
        return activations
    

    def tanh(self, x):
        return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))


if __name__ == "__main__":
    mlp = MLP(3,[3, 4, 5],1)
    inputs = np.random.rand(mlp.num_inputs)
    outputs = mlp.forward(inputs)
    print("the network input is: {}" .format(inputs)) 
    print("the network input is: {}" .format(outputs)) 



3
2
1
0
the network input is: [0.1851304  0.70187653 0.24035562]
the network input is: [-0.6848136]


In [45]:
def tanh(x):
    y = 1.0 / (1 + np.exp(-x))
    return y
x=np.array([1,3,5])
y = tanh(x)
y

array([0.73105858, 0.95257413, 0.99330715])

In [46]:
a = [3,5]
a = a + [1] 
a

[3, 5, 1]

In [13]:
import numpy as np
a = np.zeros((2,3))
a
            

array([[0., 0., 0.],
       [0., 0., 0.]])

In [19]:
import numpy as np
from random import random

class MLP():

    def __init__(self, num_inputs=3, num_hidden=[3,5], num_outputs=3):

        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i] , layers[i+1] )
            weights.append(w)
        self.weights = weights
        # print("weights: ", weights)

        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations
        # print("activations: ", activations)

        derivatives = []
        for i in range(len(layers)-1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives
        # print("derivatives: ", derivatives)

    def forward(self, inputs):
        
        activations = inputs
        self.activations[0] = activations
        for i, w in enumerate(self.weights):
            net_inputs = np.dot(activations, w)
            # print("net inputs:", net_inputs)
            activations = self.tanh(net_inputs)
            self.activations[i+1] = activations           
            # print("activations: {} ".format(activations))
            # print("final activations:", activations)
        return activations


    def back_prop(self, error):
        for i in reversed(range(len(self.derivatives))):
            activations = self.activations[i+1]
            delta = error * self.derivative(activations)
            print(delta)





    def tanh(self, x):
        return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))

    def derivative(self, x):
        return (1.0 - x**2)
   


if __name__ == "__main__":
    mlp = MLP(3,[3, 4, 5],1)
    #inputs = np.random.rand(mlp.num_inputs)
    inputs = np.array([.1, .2, .3])
    target = np.array([6])
    outputs = mlp.forward(inputs)
    error = outputs - target
    mlp.back_prop(error)
    print("the network input is: {}" .format(inputs)) 
    print("the network output is: {}" .format(outputs)) 



[-5.89725804]
[-5.53832495 -5.33909682 -4.3287109  -4.47419184 -5.83416573]
[-5.78162053 -5.78622514 -5.9279111  -5.84073707]
[-5.92575759 -5.2813184  -5.91963089]
the network input is: [0.1 0.2 0.3]
the network output is: [0.07200607]


In [7]:
a = np.array([[1,2],[3,5], [6,7], [9,0]])
b = np.array([[1,2]])
type(a)

numpy.ndarray

In [28]:
import numpy as np
import pandas as pd
df = pd.read_csv("Data_SCA.csv")
data = df.to_numpy()
print(len(data))
#train_data = data[: int(len(data)*0.7) ]
#test_data = data[int(len(data)*0.7):  ]
train_inputs =data[ : , : 2 ]
train_targets =data[ : , -1:]
train_targets
train_inputs

4898


array([[0.27, 0.36],
       [0.3 , 0.34],
       [0.28, 0.4 ],
       ...,
       [0.24, 0.19],
       [0.29, 0.3 ],
       [0.21, 0.38]])

In [25]:
data.values[:,:-1]

array([[ 0.27,  0.36, 20.7 , ...,  3.  ,  0.45,  8.8 ],
       [ 0.3 ,  0.34,  1.6 , ...,  3.3 ,  0.49,  9.5 ],
       [ 0.28,  0.4 ,  6.9 , ...,  3.26,  0.44, 10.1 ],
       ...,
       [ 0.24,  0.19,  1.2 , ...,  2.99,  0.46,  9.4 ],
       [ 0.29,  0.3 ,  1.1 , ...,  3.34,  0.38, 12.8 ],
       [ 0.21,  0.38,  0.8 , ...,  3.26,  0.32, 11.8 ]])

In [27]:
data.values[: ]

array([[ 0.27,  0.36, 20.7 , ...,  0.45,  8.8 ,  6.  ],
       [ 0.3 ,  0.34,  1.6 , ...,  0.49,  9.5 ,  6.  ],
       [ 0.28,  0.4 ,  6.9 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 0.24,  0.19,  1.2 , ...,  0.46,  9.4 ,  6.  ],
       [ 0.29,  0.3 ,  1.1 , ...,  0.38, 12.8 ,  7.  ],
       [ 0.21,  0.38,  0.8 , ...,  0.32, 11.8 ,  6.  ]])